
... ***CURRENTLY UNDER DEVELOPMENT*** ...


In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op
from datetime import datetime

# pip
import numpy as np
import xarray as xr

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.waves import GetDistribution



## Database and Site parameters

In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/nico/Projects/TESLA-kit/TeslaKit/data'
db = Database(p_data)

# set site
db.SetSite('ROI')


In [3]:
# --------------------------------------
# load data and set parameters

WVS_pts = db.Load_WAVES_partitions()  # waves partitions data

_, TCs_r1_params = db.Load_TCs_r1_hist()   # TCs historical parameters inside big radius
_, TCs_r2_params = db.Load_TCs_r2_hist()   # TCs historical parameters inside small radius

# wave families sectors
fams_sectors = [(210, 22.5), (22.5, 135)]

# date limits for TCs removal from waves data, and TC time window (hours)
tc_rm_date1 = '1979-01-01'
tc_rm_date2 = '2015-12-31'
tc_time_window = 12



## Calculate Historical Waves Families

In [4]:
# --------------------------------------
# Calculate wave families from waves partitions data and waves sectors

WVS = GetDistribution(WVS_pts, fams_sectors)
print(WVS)


<xarray.Dataset>
Dimensions:      (time: 111040)
Coordinates:
  * time         (time) datetime64[ns] 1979-01-01 ... 2016-12-31T21:00:00
Data variables:
    Hs           (time) float64 3.316 3.457 3.382 3.326 ... 2.105 2.114 2.195
    Tp           (time) float64 9.957 9.872 9.856 9.804 ... 9.907 10.4 10.71
    Dir          (time) float64 58.47 61.54 63.04 64.09 ... 61.08 59.36 58.95
    sea_Hs       (time) float64 0.3466 0.3613 0.3601 0.3554 ... 0.0607 nan 0.146
    sea_Tp       (time) float64 3.214 2.404 2.407 10.35 ... 2.952 nan 4.371
    sea_Dir      (time) float64 120.8 120.1 120.7 124.1 ... 154.0 nan 153.3
    swell_1_Hs   (time) float64 0.7837 0.7326 0.6812 ... 0.4242 0.475 0.5702
    swell_1_Tp   (time) float64 12.62 12.57 12.52 12.49 ... 12.51 12.52 12.57
    swell_1_Dir  (time) float64 4.604 4.563 4.404 3.27 ... 337.2 333.2 336.9
    swell_2_Hs   (time) float64 3.203 3.359 3.293 3.251 ... 2.054 2.05 2.11
    swell_2_Tp   (time) float64 9.892 9.741 9.719 9.626 ... 8.638 8.584 8.


## TCs: Waves  Selection 

In [5]:
# --------------------------------------
# Locate TCs and set category alongside WAVES data  

# remove TCs before 1979 and after 2015 (r1)
dds = TCs_r1_params.dmin_date.values[:]
ix = np.where((dds >= np.datetime64(tc_rm_date1)) & (dds <= np.datetime64(tc_rm_date2)))[0]
TCs_r1_params = TCs_r1_params.isel(storm=ix)

# select storms inside big circle 
storms_sel = TCs_r1_params.storm.values[:]

# add TCs category alongside WAVES data
WVS['TC_category'] = (('time',), np.empty(len(WVS.time))*np.nan)

for s in storms_sel:

    # waves at storm dates
    ss = TCs_r1_params.sel(storm=s)     
    wvs_s = WVS.sel(time = slice(ss.dmin_date, ss.last_date))
    
    # get hs_max date 
    t_hs_max = wvs_s.where(wvs_s.Hs ==  wvs_s.Hs.max(), drop=True).time.values[:][0]
        
    # hs_max time window 
    w1 = t_hs_max - np.timedelta64(tc_time_window,'h')
    w2 = t_hs_max + np.timedelta64(tc_time_window,'h')
    
    # set category alongside WAVES data
    ixs = np.where((WVS.time >= w1) & (WVS.time <= w2))[0]
    WVS['TC_category'][ixs] = ss.category

print(WVS)
    
# Store historical WAVES data
db.Save_WAVES_hist(WVS)


<xarray.Dataset>
Dimensions:      (time: 111040)
Coordinates:
  * time         (time) datetime64[ns] 1979-01-01 ... 2016-12-31T21:00:00
Data variables:
    Hs           (time) float64 3.316 3.457 3.382 3.326 ... 2.105 2.114 2.195
    Tp           (time) float64 9.957 9.872 9.856 9.804 ... 9.907 10.4 10.71
    Dir          (time) float64 58.47 61.54 63.04 64.09 ... 61.08 59.36 58.95
    sea_Hs       (time) float64 0.3466 0.3613 0.3601 0.3554 ... 0.0607 nan 0.146
    sea_Tp       (time) float64 3.214 2.404 2.407 10.35 ... 2.952 nan 4.371
    sea_Dir      (time) float64 120.8 120.1 120.7 124.1 ... 154.0 nan 153.3
    swell_1_Hs   (time) float64 0.7837 0.7326 0.6812 ... 0.4242 0.475 0.5702
    swell_1_Tp   (time) float64 12.62 12.57 12.52 12.49 ... 12.51 12.52 12.57
    swell_1_Dir  (time) float64 4.604 4.563 4.404 3.27 ... 337.2 333.2 336.9
    swell_2_Hs   (time) float64 3.203 3.359 3.293 3.251 ... 2.054 2.05 2.11
    swell_2_Tp   (time) float64 9.892 9.741 9.719 9.626 ... 8.638 8.584 8.